In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training_d = pd.read_csv("/kaggle/input/stockdataset/Stock-Dataset/Training/NFLX.csv")
testing_d = pd.read_csv("/kaggle/input/stockdataset/Stock-Dataset/Testing/NFLX.csv")

In [ ]:
training_d

In [ ]:
pip install gluonts

In [ ]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.n_beats import NBEATSEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.model.transformer import TransformerEstimator
from gluonts.model.wavenet import WaveNetEstimator
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.evaluation.backtest import make_evaluation_predictions
from datetime import datetime
import csv
import os
import subprocess
import numpy as np
import pandas as pd
#import utils.data_loader as loader

In [ ]:
training_d

In [ ]:
training_d.drop(['Open','High','Low','Adj Close','Volume'], axis = 1, inplace = True)

In [ ]:
testing_d.drop(['Open','High','Low','Adj Close','Volume'], axis = 1, inplace = True)

In [ ]:
testing_d

In [ ]:
train_start_time=training_d['Date']

In [ ]:
train_start_time

In [ ]:
train_start_time = datetime.strptime('1900-01-01 00-00-00', '%Y-%m-%d %H-%M-%S')
freq = "1D"

In [ ]:
new_df=pd.concat([training_d['Close'],testing_d['Close']],axis = 0)

In [ ]:
    train_series_full_list=[]   
    train_series_full_list.append({
            FieldName.TARGET: training_d['Close'],
            FieldName.START: pd.Timestamp(train_start_time, freq=freq)
        })

In [ ]:
    test_series_full_list=[]   
    test_series_full_list.append({
            FieldName.TARGET: new_df,
            FieldName.START: pd.Timestamp(train_start_time, freq=freq)
        })

In [ ]:
train_series_full_list

In [ ]:
test_series_full_list

In [ ]:
train_ds = ListDataset(train_series_full_list, freq=freq)
test_ds = ListDataset(test_series_full_list, freq=freq)

In [ ]:
 estimator1 = SimpleFeedForwardEstimator(freq=freq,context_length=15,prediction_length=testing_d.shape[0])

In [ ]:
 estimator2 =  DeepAREstimator(freq=freq,context_length=15,prediction_length=testing_d.shape[0])

In [ ]:
 estimator3 = NBEATSEstimator(freq=freq,context_length=15,prediction_length=testing_d.shape[0])

In [ ]:
 estimator4 =  WaveNetEstimator(freq=freq,prediction_length=testing_d.shape[0])

In [ ]:
estimator5 =  TransformerEstimator(freq=freq,context_length=15,prediction_length=testing_d.shape[0]) 

In [ ]:
 predictor1 = estimator1.train(training_data=train_ds)

In [ ]:
 predictor2 = estimator2.train(training_data=train_ds)

In [ ]:
 predictor3 = estimator3.train(training_data=train_ds)

In [ ]:
 predictor4 = estimator4.train(training_data=train_ds)

In [ ]:
 predictor5 = estimator5.train(training_data=train_ds)

In [ ]:
forecast1, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=predictor1, num_samples=100)

In [ ]:
forecast2, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=predictor2, num_samples=100)

In [ ]:
forecast3, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=predictor3, num_samples=100)

In [ ]:
forecast4, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=predictor4, num_samples=100)

In [ ]:
forecast5, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=predictor5, num_samples=100)

In [ ]:
forecasts = list(forecast1)
final_forecasts = []
for f in forecasts:
    final_forecasts.append(f.median)
y1= final_forecasts[0]    

In [ ]:
forecasts = list(forecast2)
final_forecasts = []
for f in forecasts:
    final_forecasts.append(f.median)
y2= final_forecasts[0] 

In [ ]:
forecasts = list(forecast3)
final_forecasts = []
for f in forecasts:
    final_forecasts.append(f.median)
y3= final_forecasts[0] 

In [ ]:
forecasts = list(forecast4)
final_forecasts = []
for f in forecasts:
    final_forecasts.append(f.median)
y4= final_forecasts[0] 

In [ ]:
forecasts = list(forecast5)
final_forecasts = []
for f in forecasts:
    final_forecasts.append(f.median)
y5= final_forecasts[0] 

In [ ]:
final_forecasts[0]

In [ ]:
testing_d['Close']

In [ ]:
ts_entry= list(ts_it)

In [ ]:
def SMAPE(y,yhat):
    y, yhat = np.array(y), np.array(yhat)
    new = (np.abs(yhat) + np.abs(y))
    smap=2*np.abs(y-yhat)/new
    smap_final = np.mean(smap)
    return 100*smap_final

def RMSE(y, yhat):
    y, yhat = np.array(y), np.array(yhat)
    return np.sqrt(np.mean(np.square(y - yhat)))

def MAE(y,yhat):
    y, yhat = np.array(y), np.array(yhat)
    mae = np.mean(np.abs(y - yhat))
    return mae

def msMAPE(y,yhat):
    epsilon = 0.1  
    y, yhat = np.array(y), np.array(yhat)
    comparator = np.zeros((y.shape[0],))+epsilon
    sum_1 = np.maximum(comparator, (abs(y) + abs(yhat) + epsilon))
    msmape = 2 * abs(y -yhat ) / (sum_1)
    return 100*np.mean(msmape)

def MASE(training_series, testing_series, prediction_series,S):
    y_train = np.array(training_series)
    y_test = np.array(testing_series)
    yhat = np.array(prediction_series)
    n1 = y_train.shape[0]
    d = (np.abs(np.diff(y_train,n=S)).sum())/(n1-S)
    errors = np.abs(y_test - yhat)
    return np.mean(errors)/d

In [ ]:
smape1 = SMAPE(testing_d['Close'],y1)
rmse1 = RMSE(testing_d['Close'],y1)
mae1 = MAE(testing_d['Close'],y1)
msmape1 = msMAPE(testing_d['Close'],y1)
mase1 = MASE(training_d['Close'],testing_d['Close'],y1,7)

smape2 = SMAPE(testing_d['Close'],y2)
rmse2 = RMSE(testing_d['Close'],y2)
mae2 = MAE(testing_d['Close'],y2)
msmape2 = msMAPE(testing_d['Close'],y2)
mase2 = MASE(training_d['Close'],testing_d['Close'],y2,7)

smape3 = SMAPE(testing_d['Close'],y3)
rmse3 = RMSE(testing_d['Close'],y3)
mae3 = MAE(testing_d['Close'],y3)
msmape3 = msMAPE(testing_d['Close'],y3)
mase3 = MASE(training_d['Close'],testing_d['Close'],y3,7)

smape4 = SMAPE(testing_d['Close'],y4)
rmse4 = RMSE(testing_d['Close'],y4)
mae4 = MAE(testing_d['Close'],y4)
msmape4 = msMAPE(testing_d['Close'],y4)
mase4 = MASE(training_d['Close'],testing_d['Close'],y4,7)

smape5 = SMAPE(testing_d['Close'],y5)
rmse5 = RMSE(testing_d['Close'],y5)
mae5 = MAE(testing_d['Close'],y5)
msmape5 = msMAPE(testing_d['Close'],y5)
mase5 = MASE(training_d['Close'],testing_d['Close'],y5,7)

In [ ]:
smape5 = SMAPE(testing_d['Close'],y5)
rmse5 = RMSE(testing_d['Close'],y5)
mae5 = MAE(testing_d['Close'],y5)
msmape5 = msMAPE(testing_d['Close'],y5)
mase5 = MASE(training_d['Close'],testing_d['Close'],y5,7)

In [ ]:
def get_error_metrics(model, mase, smape, rmse, mae, msmape):
    print(model + " is used, the following are result Obtained:")
    print("The mase is: " + str(mase))
    print("The smape is: " + str(smape))
    print("The rmse is: " + str(rmse))
    print("The mae is: " + str(mae))
    print("The msmape is: " + str(msmape))

In [ ]:
get_error_metrics("Simple Forward Feed Estimator", mase1, smape1, rmse1, mae1, msmape1)
get_error_metrics("Deep AR  Estimator", mase2, smape2, rmse2, mae2, msmape2)
get_error_metrics("NBEATS Estimator", mase3, smape3, rmse3, mae3, msmape3)
get_error_metrics("Wavenet Estimator", mase4, smape4, rmse4, mae4, msmape4)


In [ ]:
get_error_metrics("Transformer Estimator", mase5, smape5, rmse5, mae5, msmape5)